# 🚀 PHASE 2: ADVANCED DEEP LEARNING MODELS
## Hedge Fund Tier Trading System - GPU Training

### 🎯 Goals:
- Train Transformer model (attention-based)
- Train CNN-LSTM model (hybrid approach)
- Create ensemble predictions
- Target: 96%+ accuracy (better than 95.3% basic model)

### ⏱️ Timeline: Hours 4-8
### 💰 Investment: Free Colab GPU (testing first)


In [ ]:
# Check GPU availability and setup
import torch
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("🚀 PHASE 2: ADVANCED DEEP LEARNING MODELS")
print("=" * 60)
print(f"🔧 GPU Status: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU Name: {torch.cuda.get_device_name()}")
    print(f"🔧 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    device = torch.device('cuda')
    print(f"🚀 Using GPU for training!")
else:
    print(f"❌ No GPU - enable GPU in Runtime settings")
    device = torch.device('cpu')

print(f"📊 Device: {device}")
print(f"⏰ Started at: {datetime.now().strftime('%H:%M:%S')}")

In [ ]:
# Install required packages
print(f"📦 Installing packages...")
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers scikit-learn pandas numpy matplotlib seaborn yfinance ta
print(f"✅ Packages installed!")

In [ ]:
# Load ALL our Tiingo data (46 stocks)
import pandas as pd
import numpy as np
import glob
import os
from sklearn.preprocessing import StandardScaler

print(f"📈 Loading ALL our Tiingo data (46 stocks)...")

def load_tiingo_data(ticker):
    """Load Tiingo CSV data - supports all file naming patterns and locations"""
    try:
        # Search in multiple possible locations
        search_paths = [
            f"data/cache/tiingo/{ticker}*.csv",
            f"data/cache/{ticker}*.csv",
            f"{ticker}*.csv"  # Current directory as fallback
        ]
        
        csv_files = []
        for path in search_paths:
            found = glob.glob(path)
            csv_files.extend(found)
        
        if not csv_files:
            print(f"❌ No CSV files found for {ticker}")
            return None
        
        # Use the first (or best) matching file
        file_path = csv_files[0]
        print(f"📁 Found {ticker} file: {os.path.basename(file_path)}")
        
        df = pd.read_csv(file_path)
        
        # Handle date column
        date_col = None
        for col in df.columns:
            if col.lower() == 'date':
                date_col = col
                break
            elif col.lower() == 'datetime':
                date_col = col
                break
        
        if date_col:
            df[date_col] = pd.to_datetime(df[date_col])
            df.set_index(date_col, inplace=True)
        elif 'Date' in df.columns:
            df['Date'] = pd.to_datetime(df['Date'])
            df.set_index('Date', inplace=True)
        
        # Map columns to standard names
        col_mapping = {}
        for col in df.columns:
            col_lower = col.lower()
            if 'close' in col_lower and 'Close' not in df.columns:
                col_mapping['Close'] = col
            elif 'open' in col_lower and 'Open' not in df.columns:
                col_mapping['Open'] = col
            elif 'high' in col_lower and 'High' not in df.columns:
                col_mapping['High'] = col
            elif 'low' in col_lower and 'Low' not in df.columns:
                col_mapping['Low'] = col
            elif 'volume' in col_lower and 'Volume' not in df.columns:
                col_mapping['Volume'] = col
            elif 'adj close' in col_lower and 'Close' not in df.columns:
                col_mapping['Close'] = col
        
        if col_mapping:
            df = df.rename(columns=col_mapping)
        
        return df
        
    except Exception as e:
        print(f"❌ Error loading {ticker}: {e}")
        return None

# First, let's see what CSV files we actually have
print(f"🔍 Scanning for available CSV files...")
all_csv_files = []
all_csv_files.extend(glob.glob("data/cache/tiingo/*.csv"))
all_csv_files.extend(glob.glob("data/cache/*.csv"))

print(f"📁 Found {len(all_csv_files)} CSV files:")
for file in sorted(all_csv_files)[:10]:  # Show first 10
    print(f"   {os.path.basename(file)}")
if len(all_csv_files) > 10:
    print(f"   ... and {len(all_csv_files) - 10} more")

# Extract tickers from available files
available_tickers = set()
for file in all_csv_files:
    ticker = os.path.basename(file).split('_')[0]  # Get ticker before first underscore
    available_tickers.add(ticker)

print(f"\n📊 Available tickers from files: {sorted(list(available_tickers))}")

# ALL 46 STOCKS we want to load
all_stocks = [
    # Technology (9 stocks)
    'AAPL', 'MSFT', 'GOOGL', 'META', 'NVDA', 'AMD', 'INTC', 'CSCO', 'ORCL',
    
    # Financial (6 stocks)
    'JPM', 'V', 'AXP', 'BAC', 'WFC', 'GS',
    
    # Healthcare (5 stocks)
    'JNJ', 'UNH', 'PFE', 'MRK', 'ABT',
    
    # Consumer (12 stocks)
    'AMZN', 'TSLA', 'DIS', 'WMT', 'PG', 'HD', 'MCD', 'NKE', 'BABA', 'LULU', 'EL', 'CALM',
    
    # Industrial (7 stocks)
    'CAT', 'GE', 'HON', 'AAL', 'DAL', 'UAL', 'LUV',
    
    # Energy (2 stocks)
    'XOM', 'CVX',
    
    # ETFs (5 stocks)
    'QQQ', 'SPY', 'VXX', 'KLAC', 'EIS'
]

print(f"\n📊 Attempting to load {len(all_stocks)} stocks...")

stock_data = {}
successful_loads = 0
failed_loads = []

for ticker in all_stocks:
    data = load_tiingo_data(ticker)
    if data is not None:
        stock_data[ticker] = data
        successful_loads += 1
        print(f"✅ {ticker}: {len(data)} days loaded")
    else:
        failed_loads.append(ticker)
        print(f"❌ {ticker}: Failed to load")

print(f"\n📊 LOADING SUMMARY:")
print(f"✅ Successfully loaded: {successful_loads}/{len(all_stocks)} stocks")
print(f"❌ Failed to load: {len(failed_loads)} stocks")

if failed_loads:
    print(f"❌ Failed stocks: {failed_loads}")

# Use GOOGL as our primary stock for training (best performer)
if 'GOOGL' in stock_data:
    googl_data = stock_data['GOOGL'].copy()
    # Reset index to get Date as a column
    googl_data.reset_index(inplace=True)
    print(f"\n🎯 Using GOOGL for training: {len(googl_data)} days")
    print(f"? Columns: {list(googl_data.columns)}")
    
    # Check if we have a Date column after reset_index
    if 'Date' in googl_data.columns:
        print(f"?📅 Period: {googl_data['Date'].min().date()} to {googl_data['Date'].max().date()}")
    else:
        print(f"⚠️ Date column not found, available columns: {list(googl_data.columns)}")
else:
    print(f"\n❌ GOOGL data not found!")
    # Use first available stock
    if stock_data:
        first_stock = list(stock_data.keys())[0]
        googl_data = stock_data[first_stock].copy()
        googl_data.reset_index(inplace=True)
        print(f"🔄 Using {first_stock} instead: {len(googl_data)} days")

print(f"\n🎉 PORTFOLIO STATUS:")
print(f"📊 Total stocks available: {len(stock_data)}")
print(f"📈 Stock list: {sorted(list(stock_data.keys()))}")

# Display sector breakdown
sectors = {
    'Technology': ['AAPL', 'MSFT', 'GOOGL', 'META', 'NVDA', 'AMD', 'INTC', 'CSCO', 'ORCL'],
    'Financial': ['JPM', 'V', 'AXP', 'BAC', 'WFC', 'GS'],
    'Healthcare': ['JNJ', 'UNH', 'PFE', 'MRK', 'ABT'],
    'Consumer': ['AMZN', 'TSLA', 'DIS', 'WMT', 'PG', 'HD', 'MCD', 'NKE', 'BABA', 'LULU', 'EL', 'CALM'],
    'Industrial': ['CAT', 'GE', 'HON', 'AAL', 'DAL', 'UAL', 'LUV'],
    'Energy': ['XOM', 'CVX'],
    'ETFs': ['QQQ', 'SPY', 'VXX', 'KLAC', 'EIS']
}

print(f"\n🏢 SECTOR BREAKDOWN:")
for sector, stocks in sectors.items():
    loaded = [s for s in stocks if s in stock_data]
    print(f"   {sector}: {len(loaded)}/{len(stocks)} loaded - {loaded}")

print(f"\n✅ Data loading complete! Ready for feature engineering...")

In [ ]:
# Create enhanced features
def create_enhanced_features(df):
    features = pd.DataFrame(index=df.index)
    
    # Price-based features
    for period in [5, 10, 20, 50]:
        features[f'close_ma_{period}'] = df['Close'].rolling(period).mean()
        features[f'close_std_{period}'] = df['Close'].rolling(period).std()
        features[f'close_bb_position_{period}'] = (df['Close'] - features[f'close_ma_{period}']) / features[f'close_std_{period}']
        features[f'close_ma_ratio_{period}'] = df['Close'] / features[f'close_ma_{period}']
    
    # Returns
    for period in [1, 5, 10, 20]:
        features[f'return_{period}'] = df['Close'].pct_change(period)
        features[f'return_abs_{period}'] = abs(df['Close'].pct_change(period))
    
    # RSI
    for period in [14, 30]:
        delta = df['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        features[f'rsi_{period}'] = 100 - (100 / (1 + rs))
    
    # MACD
    ema_12 = df['Close'].ewm(span=12).mean()
    ema_26 = df['Close'].ewm(span=26).mean()
    features['macd'] = ema_12 - ema_26
    features['macd_signal'] = features['macd'].ewm(span=9).mean()
    
    # Volume
    features['volume_ratio'] = df['Volume'] / df['Volume'].rolling(20).mean()
    
    return features.dropna()

features = create_enhanced_features(googl_data)
print(f"✅ Enhanced features created: {features.shape[1]} features")

In [ ]:
# Create sequences for deep learning
def create_sequences(features, target, sequence_length=30):
    X, y = [], []
    for i in range(sequence_length, len(features)):
        X.append(features.iloc[i-sequence_length:i].values)
        y.append(target.iloc[i])
    return np.array(X), np.array(y)

# Prepare target and sequences
target = googl_data['Close'].shift(-1)
features = features.dropna()
target = target.dropna()
common_idx = features.index.intersection(target.index)
features = features.loc[common_idx]
target = target.loc[common_idx]

X, y = create_sequences(features, target, 30)

# Proper chronological split: 70% train, 15% val, 15% test
train_idx = int(0.7 * len(X))
val_idx = int(0.85 * len(X))

X_train, X_val, X_test = X[:train_idx], X[train_idx:val_idx], X[val_idx:]
y_train, y_val, y_test = y[:train_idx], y[train_idx:val_idx], y[val_idx:]

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_val_scaled = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)
X_test_scaled = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

print(f"✅ Sequences created:")
print(f"   Train: {X_train_scaled.shape}")
print(f"   Val: {X_val_scaled.shape}")  
print(f"   Test: {X_test_scaled.shape}")
print(f"   Target range: ${y_train.min():.2f} - ${y_train.max():.2f}")

In [ ]:
# Define deep learning models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class TransformerModel(nn.Module):
    def __init__(self, input_size, d_model=256, n_heads=8, num_layers=4, dropout=0.1):
        super().__init__()
        self.input_projection = nn.Linear(input_size, d_model)
        self.positional_encoding = nn.Parameter(torch.randn(1000, d_model) * 0.02)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=n_heads, dim_feedforward=512, dropout=dropout, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers)
        self.output_projection = nn.Sequential(
            nn.Linear(d_model, 128), 
            nn.ReLU(), 
            nn.Dropout(dropout), 
            nn.Linear(128, 64), 
            nn.ReLU(), 
            nn.Linear(64, 1)
        )
        self._init_weights()
    
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        x = self.input_projection(x)
        seq_len = x.size(1)
        x = x + self.positional_encoding[:seq_len]
        x = self.transformer(x)
        x = x[:, -1, :]
        x = self.output_projection(x)
        return x

class CNNLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size=256, num_layers=2, dropout=0.1):
        super().__init__()
        # CNN layers for feature extraction
        self.conv1d_1 = nn.Conv1d(input_size, 128, kernel_size=3, padding=1)
        self.conv1d_2 = nn.Conv1d(128, 256, kernel_size=3, padding=1)
        self.conv1d_3 = nn.Conv1d(256, 512, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.dropout = nn.Dropout(dropout)
        
        # LSTM for temporal patterns
        self.lstm = nn.LSTM(512, hidden_size, num_layers, batch_first=True, dropout=dropout)
        
        # Output layers
        self.output = nn.Sequential(
            nn.Linear(hidden_size, 128), 
            nn.ReLU(), 
            nn.Dropout(dropout), 
            nn.Linear(128, 64), 
            nn.ReLU(), 
            nn.Linear(64, 1)
        )
        self._init_weights()
    
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
    
    def forward(self, x):
        # CNN feature extraction
        x = x.transpose(1, 2)  # (batch, features, seq_len)
        x = torch.relu(self.conv1d_1(x))
        x = self.pool(x)
        x = self.dropout(x)
        x = torch.relu(self.conv1d_2(x))
        x = self.pool(x)
        x = self.dropout(x)
        x = torch.relu(self.conv1d_3(x))
        x = self.dropout(x)
        
        # LSTM processing
        x = x.transpose(1, 2)  # (batch, seq_len, features)
        lstm_out, (hidden, cell) = self.lstm(x)
        final_output = lstm_out[:, -1, :]  # Use last output
        
        # Final prediction
        output = self.output(final_output)
        return output

print(f"🤖 Improved models defined successfully!")

In [ ]:
# Training and evaluation functions
def train_model(model, X_train, y_train, X_val, y_val, epochs=50, batch_size=32, learning_rate=5e-5):
    model = model.to(device)
    
    # Convert to tensors
    X_train_tensor = torch.FloatTensor(X_train).to(device)
    y_train_tensor = torch.FloatTensor(y_train).to(device)
    X_val_tensor = torch.FloatTensor(X_val).to(device)
    y_val_tensor = torch.FloatTensor(y_val).to(device)
    
    # Create data loaders
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    # Optimizer with conservative settings
    optimizer = optim.AdamW(
        model.parameters(), 
        lr=learning_rate,  # Much lower learning rate
        weight_decay=1e-4,  # Add regularization
        betas=(0.9, 0.999),
        eps=1e-8
    )
    
    # Learning rate scheduler
    scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer, 
        max_lr=learning_rate,
        epochs=epochs,
        steps_per_epoch=len(train_loader),
        pct_start=0.1,
        anneal_strategy='cos'
    )
    
    criterion = nn.MSELoss()
    best_val_loss = float('inf')
    train_losses = []
    val_losses = []
    
    print(f"🧠 Training on {device}...")
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        num_batches = 0
        
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(batch_X).squeeze()
            loss = criterion(outputs, batch_y)
            
            # Backward pass with gradient clipping
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
            optimizer.step()
            scheduler.step()
            
            train_loss += loss.item()
            num_batches += 1
        
        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val_tensor).squeeze()
            val_loss = criterion(val_outputs, y_val_tensor).item()
        
        avg_train_loss = train_loss / num_batches
        train_losses.append(avg_train_loss)
        val_losses.append(val_loss)
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_model.pth')
        
        # Print progress
        if epoch % 10 == 0:
            print(f"   Epoch {epoch:3d}: Train Loss: {avg_train_loss:.6f}, Val Loss: {val_loss:.6f}")
            
            # Check for overfitting
            if val_loss > avg_train_loss * 3:
                print(f"   ⚠️ Overfitting detected!")
    
    # Load best model
    model.load_state_dict(torch.load('best_model.pth'))
    print(f"✅ Training completed! Best val loss: {best_val_loss:.6f}")
    return model, train_losses, val_losses

def evaluate_model(model, X_test, y_test):
    model.eval()
    X_test_tensor = torch.FloatTensor(X_test).to(device)
    with torch.no_grad():
        predictions = model(X_test_tensor).squeeze().cpu().numpy()
    
    mse = np.mean((predictions - y_test) ** 2)
    rmse = np.sqrt(mse)
    
    # Better accuracy calculation
    y_mean = np.mean(y_test)
    accuracy = max(0, 1 - (rmse / y_mean))  # Ensure non-negative
    
    # Direction accuracy
    direction_correct = np.mean(np.sign(predictions[:-1]) == np.sign(y_test[1:]))
    
    return {
        'predictions': predictions, 
        'rmse': rmse, 
        'accuracy': accuracy,
        'direction_accuracy': direction_correct
    }

print(f"🎯 Improved training functions ready!")

In [ ]:
# Train Transformer model
print(f"🤖 Training Transformer model...")
transformer_model = TransformerModel(
    input_size=X_train_scaled.shape[-1], 
    d_model=256, 
    n_heads=8, 
    num_layers=4, 
    dropout=0.1
)
transformer_model, transformer_train_losses, transformer_val_losses = train_model(
    transformer_model, 
    X_train_scaled, y_train, 
    X_val_scaled, y_val, 
    epochs=50, 
    batch_size=32,
    learning_rate=5e-5
)
transformer_results = evaluate_model(transformer_model, X_test_scaled, y_test)
print(f"📊 Transformer Results:")
print(f"   Accuracy: {transformer_results['accuracy']:.1%}")
print(f"   Direction Accuracy: {transformer_results['direction_accuracy']:.1%}")
print(f"   RMSE: ${transformer_results['rmse']:.2f}")

In [ ]:
# Train CNN-LSTM model
print(f"🤖 Training CNN-LSTM model...")
cnn_lstm_model = CNNLSTMModel(
    input_size=X_train_scaled.shape[-1], 
    hidden_size=256, 
    num_layers=2, 
    dropout=0.1
)
cnn_lstm_model, cnn_train_losses, cnn_val_losses = train_model(
    cnn_lstm_model, 
    X_train_scaled, y_train, 
    X_val_scaled, y_val, 
    epochs=50, 
    batch_size=32,
    learning_rate=5e-5
)
cnn_lstm_results = evaluate_model(cnn_lstm_model, X_test_scaled, y_test)
print(f"📊 CNN-LSTM Results:")
print(f"   Accuracy: {cnn_lstm_results['accuracy']:.1%}")
print(f"   Direction Accuracy: {cnn_lstm_results['direction_accuracy']:.1%}")
print(f"   RMSE: ${cnn_lstm_results['rmse']:.2f}")

In [ ]:
# Create ensemble and compare
ensemble_predictions = (transformer_results['predictions'] + cnn_lstm_results['predictions']) / 2
ensemble_rmse = np.sqrt(np.mean((ensemble_predictions - y_test) ** 2))
ensemble_accuracy = max(0, 1 - (ensemble_rmse / np.mean(y_test)))
ensemble_direction_accuracy = np.mean(np.sign(ensemble_predictions[:-1]) == np.sign(y_test[1:]))

basic_accuracy = 0.953  # From Phase 1
improvement = ensemble_accuracy - basic_accuracy

print(f"\n🎯 PERFORMANCE COMPARISON:")
print(f"   Basic Model (Random Forest): {basic_accuracy:.1%}")
print(f"   Transformer: {transformer_results['accuracy']:.1%}")
print(f"   CNN-LSTM: {cnn_lstm_results['accuracy']:.1%}")
print(f"   Deep Learning Ensemble: {ensemble_accuracy:.1%}")
print(f"   Improvement: {improvement:+.1%}")

print(f"\n📊 DIRECTION ACCURACY:")
print(f"   Transformer: {transformer_results['direction_accuracy']:.1%}")
print(f"   CNN-LSTM: {cnn_lstm_results['direction_accuracy']:.1%}")
print(f"   Ensemble: {ensemble_direction_accuracy:.1%}")

print(f"\n💰 PREDICTION ERROR:")
print(f"   Transformer RMSE: ${transformer_results['rmse']:.2f}")
print(f"   CNN-LSTM RMSE: ${cnn_lstm_results['rmse']:.2f}")
print(f"   Ensemble RMSE: ${ensemble_rmse:.2f}")

if ensemble_accuracy > basic_accuracy:
    print(f"\n🏆 SUCCESS! Deep learning ensemble is better!")
    print(f"   📈 Improvement: {improvement:+.1%}")
else:
    print(f"\n⚠️  Basic model still performs better")
    print(f"   📉 Gap: {basic_accuracy - ensemble_accuracy:.1%}")

# Plot training curves if you want to visualize
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(transformer_train_losses, label='Transformer Train')
plt.plot(transformer_val_losses, label='Transformer Val')
plt.title('Transformer Training')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(cnn_train_losses, label='CNN-LSTM Train')
plt.plot(cnn_val_losses, label='CNN-LSTM Val')
plt.title('CNN-LSTM Training')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

print(f"\n🎉 PHASE 2 COMPLETE!")
print(f"⏰ Finished at: {datetime.now().strftime('%H:%M:%S')}")
print(f"🚀 Ready for Phase 3: Alternative Data Integration")